# Using Monte Carlo Simulation to Determine the Optimal Portfolio of Stocks

If you have tried investing in the stock market, then you are most likely faced with multiple investment decisions such as "which stock to choose", "which industry to focus on" and "how much should you allocate to each stock".
Fortunately, Harry Markowitz provided an answer to the last question which is also considered as one of the most difficult problems in investing: portfolio security selection. 

His Moden Portfolio Theory (MPT) won him a Nobel Prize and introduced the ideas of portfolio investing and how securities' risks and correlations impact the portfolio as a whole.
So you might think that there are many ways to answer the question "How much should I invest in a stock", but optimization theories and Markowitz's work would tell you that there is only one correct answer.
There are two ways to solve this problem:

1.  **Analytically** -  which uses linear algebra and matrix operations to arrive at the optimal portfolio.

2.  **Computationally** -  with the use of computers to crunch possible permutations of the portfolio. The optimal portfolio would be the one with the highest return per risk portfolio.

Note that in portfolio optimization, what we optimize is that of the weights or the allocation, given a list of possible investments. 

To get our stock data, we will employ the investpy package.

## PRELIMINARIES

In [13]:
import numpy as np
import pandas as pd
import hvplot.pandas  # noqa

#For Monte Carlo
import random

#Visualization
import holoviews as hv

from tqdm import tqdm

#Historical Data
import investpy

## STEP 1: GET HISTORICAL DATA FOR THE STOCKS OF YOUR CHOICE

We get the stock data, to track the volatility of the market. Ideally, we want a time period that covers the full business cycle, from trough, recession, expansion, and peak.

To begin, specify the stock and past trading dates you are looking at. The past trading dates will provide us the riskiness of the stocks through the standard deviation of their returns.

In [2]:
stocks = ['MEG','CEB', 'BDO','ALI', 'MER', 'AC', 'JGS', 'URC', 
          'JFC', 'SEVN', 'BMM', 'CAT', 'FGEN', 'MAXS', 'PGOLD']

## STEP 2: CALCULATE THE RETURNS FOR THESE STOCKS

To calculate the return, specify the time period in question. This is critical as the choice of the time period may coincide with periods of high growth or periods of depressed growth for the equity market.

For now, let us assume that the correct time period that will represent the expected return for the coming year is given by the following:

In [4]:
begin_date = "01/01/2014"
end_date = "29/12/2019"

In [6]:
def generate_stock_returns(stocks_list, begin_date, end_date):
    
    prices = pd.DataFrame()
    
    for stock in stocks_list:
        df_ = investpy.get_stock_historical_data(stock=stock,
                                                country='Philippines',
                                                from_date=begin_date,
                                                to_date=end_date).Close
        df_.rename(stock, inplace=True)                                             
        df_.columns = [stock]
        prices = pd.concat([prices, df_],axis=1)
        prices.index.name = "Date"
    return prices

In [7]:
prices = generate_stock_returns(stocks, begin_date, end_date)

In [8]:
prices

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN,MAXS,PGOLD
Date,,,,,,,,,,,,,,,
2014-01-02,3.31,48.712,71.00,25.50,256.00,525.5,39.000,115.88,175.9,NaN,NaN,NaN,14.02,7.55,38.899
2014-01-03,3.31,49.109,70.45,25.25,254.20,520.5,38.850,116.86,172.0,98.0,NaN,NaN,14.14,7.65,39.098
2014-01-06,3.41,49.209,71.80,25.50,260.00,525.5,39.500,117.26,175.0,97.0,NaN,NaN,14.28,7.60,38.899
2014-01-07,3.40,48.960,71.85,25.60,257.00,524.0,38.200,113.43,172.0,95.0,NaN,NaN,14.36,7.55,38.999
2014-01-08,3.38,50.650,72.55,26.10,263.00,530.5,39.200,116.57,169.5,95.0,NaN,NaN,14.86,7.32,39.098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-19,4.10,90.700,154.80,45.40,304.63,755.0,73.333,141.04,216.0,148.0,NaN,17.54,23.50,12.10,40.200
2019-12-20,4.00,90.400,155.40,46.00,279.97,779.5,76.286,142.03,213.0,NaN,NaN,17.52,22.90,12.10,39.950
2019-12-23,4.03,88.900,157.00,47.25,293.28,779.0,77.810,149.82,220.0,145.5,NaN,17.52,23.75,12.12,39.950


To get the return, we simply use the following code:

### Returns Calculation

In [10]:
returns = prices.pct_change()
returns.head()

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN,MAXS,PGOLD
Date,,,,,,,,,,,,,,,
2014-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03 00:00:00,0.000000,0.008150,-0.007746,-0.009804,-0.007031,-0.009515,-0.003846,0.008457,-0.022172,NaN,NaN,NaN,0.008559,0.013245,0.005116
2014-01-06 00:00:00,0.030211,0.002036,0.019163,0.009901,0.022817,0.009606,0.016731,0.003423,0.017442,-0.010204,NaN,NaN,0.009901,-0.006536,-0.005090
2014-01-07 00:00:00,-0.002933,-0.005060,0.000696,0.003922,-0.011538,-0.002854,-0.032911,-0.032662,-0.017143,-0.020619,NaN,NaN,0.005602,-0.006579,0.002571
2014-01-08 00:00:00,-0.005882,0.034518,0.009743,0.019531,0.023346,0.012405,0.026178,0.027682,-0.014535,0.000000,NaN,NaN,0.034819,-0.030464,0.002539


### Covariance Matrix

In [11]:
cov = returns.cov()
cov.head()

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN,MAXS,PGOLD
MEG,0.000435,0.000086,0.000106,0.000141,0.000058,0.000119,0.000138,0.000091,0.000081,-0.000011,0.000009,0.000031,0.000077,0.000086,0.000090
CEB,0.000086,0.000555,0.000062,0.000059,0.000022,0.000048,0.000070,0.000055,0.000032,0.000002,0.000018,0.000023,0.000063,0.000078,0.000044
BDO,0.000106,0.000062,0.000232,0.000096,0.000050,0.000078,0.000107,0.000093,0.000063,0.000005,0.000015,0.000005,0.000046,0.000042,0.000055
ALI,0.000141,0.000059,0.000096,0.000303,0.000052,0.000124,0.000129,0.000109,0.000087,0.000016,0.000027,0.000005,0.000070,0.000077,0.000061
MER,0.000058,0.000022,0.000050,0.000052,0.000193,0.000044,0.000071,0.000055,0.000043,0.000002,0.000030,0.000023,0.000031,0.000017,0.000027


## STEP 3: INITIALIZE THE WEIGHTS AND FUNCTION FOR THE CALCULATION OF METRICS

As we are trying to optimize the portfolio by altering the weight allocation, let's initialize our weights and calculate the initial metrics using our customized function below.

It is important that the weights chosen come from a uniform distribution from 0 to 1. Luckily, random.random of numpy draws from a continuous, uniform distribution.

In [33]:
np.random.seed(10) #for replicability
weights = np.random.random(len(stocks))
weights /= np.sum(weights)
weights

array([0.10892225, 0.00293049, 0.08948081, 0.10574254, 0.0703968 ,
       0.03174472, 0.0279695 , 0.10739855, 0.02388103, 0.01247493,
       0.09678327, 0.1346337 , 0.00055756, 0.07232937, 0.11475449])

Note that we normalize our weights to equal to 1 or 100% allocation.

### PORTFOLIO RETURNS

Because what we have are daily returns of stocks that we have calculated, we then proceed to annualize these by multiplying the average daily returns with the number of trading days in a year which is 252.

In [22]:
rp = (returns.mean()*252)@weights 
rp

0.2115677224809396

### PORTFOLIO VARIANCE

In [ ]:
Since we have the daily volatility as well, we proceed to annualize these by multiplying them with 252. The formula for the Covariance is given as:

In [23]:
#Portfolio Variance
port_var = weights@(cov*252)@weights 
port_var

0.04448360793364385

### SHARPE RATIO

In [24]:
#Sharpe Ratio
rf = 0.02 #risk-free rate
sharpe = (rp-rf)/np.sqrt(port_var)
sharpe

0.9082854011482231

In [25]:
len(weights)

15

## FUNCTION FOR USE

In [30]:
def portfolio_metrics(weights, index='Trial'):
    
    '''
    This function generates the relative performance metrics that will be reported and will be used
    to find the optimal weights.
    
    Parameters:
    weights: initialized weights or optimal weights for performance reporting
    
    '''   
    
    rp = (returns.mean()*252)@weights 
    port_var = weights@(cov*252)@weights
    sharpe = (rp-rf)/np.sqrt(port_var)
    df = pd.DataFrame({"Expected Return": rp,
                       "Portfolio Variance":port_var,
                       'Portfolio Std': np.sqrt(port_var),
                       'Sharpe Ratio': sharpe}, index=[index])
    return df

portfolio_metrics(weights)

,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
Trial,0.211568,0.044484,0.210911,0.908285


## STEP 4:  MONTE CARLO SIMULATION

In [92]:
np.random.seed(42)
portfolios = pd.DataFrame(columns=[*stocks, "Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"])

for i in range(10000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    portfolios.loc[i, stocks] = weights
    metrics = portfolio_metrics(weights,i)
    portfolios.loc[i, ["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]] = \
    metrics.loc[i,["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]]
    
portfolios

,MEG,CEB,BDO,ALI,MER,AC,JGS,URC,JFC,SEVN,BMM,CAT,FGEN,MAXS,PGOLD,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
0,0.0504875,0.128155,0.0986718,0.0806984,0.0210311,0.0210279,0.0078296,0.116759,0.0810295,0.0954473,0.00277476,0.130743,0.112212,0.028623,0.0245098,0.18796,0.033259,0.18237,0.920983
1,0.0319551,0.053009,0.0914297,0.075259,0.0507416,0.106605,0.0243044,0.0509012,0.0638322,0.0794623,0.136803,0.0347897,0.0895965,0.103218,0.00809318,0.207465,0.037214,0.192909,0.971778
2,0.0878919,0.0246693,0.00941084,0.137273,0.139695,0.116949,0.0440677,0.01413,0.0989862,0.0636757,0.0176549,0.0716359,0.0049749,0.131549,0.037437,0.161923,0.0288528,0.169861,0.835521
3,0.0830258,0.0390629,0.0651737,0.0685125,0.0231655,0.121506,0.0971379,0.117736,0.112138,0.0749275,0.115527,0.0110897,0.0245601,0.00566778,0.0407697,0.16878,0.0329312,0.18147,0.819864
4,0.0541899,0.0378318,0.115544,0.049739,0.0391682,0.0756634,0.0196478,0.111843,0.0103939,0.137593,0.107667,0.0277052,0.0007699,0.113693,0.0985509,0.188476,0.0335834,0.183258,0.919339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0548557,0.134488,0.0281449,0.135753,0.106708,0.0378455,0.00124095,0.0617432,0.0874902,0.0157542,0.00926777,0.00650883,0.110645,0.11238,0.097175,0.130205,0.0249873,0.158074,0.697177
9996,0.100939,0.0488678,0.0167991,0.0190937,0.0442761,0.0526929,0.127771,0.0530304,0.0613739,0.060498,0.102524,0.00412233,0.0629317,0.122274,0.122806,0.1714,0.0329578,0.181543,0.833961
9997,0.111121,0.0602974,0.0377234,0.0136849,0.072362,0.128696,0.0523635,0.0771608,0.0493127,0.0605267,0.0210506,0.104214,0.00740433,0.146374,0.0577088,0.181854,0.0343108,0.185232,0.873792
9998,0.0463544,0.100703,0.154131,0.150918,0.157975,0.0160841,0.0386961,0.0125296,0.0158804,0.0321185,0.0205376,0.0566754,0.0697656,0.0579058,0.0697251,0.16193,0.0233611,0.152843,0.9286


In [99]:
optimal_portfolio = portfolios[portfolios["Sharpe Ratio"]==portfolios["Sharpe Ratio"].max()].T

In [100]:
optimal_portfolio 

,9727
MEG,0.0160691
CEB,0.106387
BDO,0.100044
ALI,0.0932814
MER,0.0145209
AC,0.0627358
JGS,0.0542231
URC,0.0275573
JFC,0.0795493
SEVN,0.109194


In [142]:
opt_weights = portfolios[portfolios["Sharpe Ratio"]==portfolios["Sharpe Ratio"].max()].to_numpy()[0][0:15]

In [143]:
len(opt_weights)

15

In [144]:
def optimal_portfolio_weights(weights):
    df = pd.DataFrame({"Stock": stocks,
                       "Weights": weights})
    df.Weights = df.Weights.map(lambda x: '{:.2%}'.format(x))
    return df

optimal_portfolio_weights(opt_weights)

,Stock,Weights
0,MEG,1.61%
1,CEB,10.64%
2,BDO,10.00%
3,ALI,9.33%
4,MER,1.45%
5,AC,6.27%
6,JGS,5.42%
7,URC,2.76%
8,JFC,7.95%
9,SEVN,10.92%
